In [25]:
import requests
import re
from lxml import etree
import time
from pyecharts.charts import Bar, Pie, Grid
from pyecharts import options as opts
# 内置主题类型可查看 pyecharts.globals.ThemeType
from pyecharts.globals import ThemeType
import pandas as pd
import numpy as np

In [29]:
#BOSS直聘(前端岗位爬取)(10页)
#https://movie.douban.com/review/best/?start=0  通过start=0参数搞
#页面上提取的时候大多数是文字型描述，比如3-5年  要提取数字并统计的话要使用re
#存储参数
save_path = 'E:/BOSS直聘爬取/' #大路径
save_name = 'BOSS直聘(前端岗位爬取)' #项目名
save_folder = ['数据表','数据图'] #选择存储的目录类型表

#爬虫控制参数
delay_num = 1  #延迟爬取间隔时间

#结束语
end_said = '已经爬取完毕'

url = 'https://www.zhipin.com/c100010000/?query=web%E5%89%8D%E7%AB%AF&page=1&ka=page-1'
params = { 
    'query': 'web前端',
    'page': '1',
    'ka': 'page-1',
}
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'cookie': 'id=sem_pz_bdpc_dasou_title; __g=sem_pz_bdpc_dasou_title; Hm_lvt_194df3105ad7148dcf2b98a91b5e727a=1606211324; lastCity=100010000; __fid=aee539a4b29b16d74147e7b56ce106fd; __l=l=%2Fwww.zhipin.com%2Fc100010000%2F%3Fquery%3Dweb%25E5%2589%258D%25E7%25AB%25AF%26page%3D1%26ka%3Dpage-1&r=https%3A%2F%2Fwww.baidu.com%2Fother.php%3Fsc.a00000K5L13vztlb3a7W-KT5C3N7sg_jI8T0vU7EBcIHoY05YblIurZP9e8h44LnauE2Z6BvImWgeIKYb0qGimPlSYHOL_1tWSN9kPpbxwwYCnaq1tnKzfZnobTE1_5foWgK7P17Dz4HYwaNRp8D3fBi1kQ2uk9Ova9Xqwi01116sbOHlIqZ1bQUre3T-CUrKp_CHL6xGaACy-zq4yXgLXgNFu_D.7D_NR2Ar5Od663rj6t8AGSPticrtXFBPrM-kt5QxIW94UhmLmry6S9wiGyAp7BEIu80.TLFWgv-b5HDkrfK1ThPGujYknHb0THY0IAYqmhq1TzPaHD7riz5y_Tp30ZN1ugFxIZ-suHYs0A7bgLw4TARqnsKLULFb5yFETL5y_Tp30ZFWIWYk0ZNzU7qGujYkPHD4Pj01PjD10Addgv-b5HDzP1mLP10s0AdxpyfqnH0vrjTsPWR0UgwsU7qGujYknHR1P0KsI-qGujYs0A-bm1dcHbc0mLFW5HndP1TL%26ck%3D1880.1.70.382.153.386.160.68%26dt%3D1606211283%26wd%3Dboss%2B%25E7%259B%25B4%25E8%2581%2598%26tpl%3Dtpl_11534_22836_18980%26l%3D1519403413%26us%3DlinkName%253D%2525E6%2525A0%252587%2525E5%252587%252586%2525E5%2525A4%2525B4%2525E9%252583%2525A8-%2525E4%2525B8%2525BB%2525E6%2525A0%252587%2525E9%2525A2%252598%2526linkText%253DBOSS%2525E7%25259B%2525B4%2525E8%252581%252598%2525E2%252580%252594%2525E2%252580%252594%2525E6%252589%2525BE%2525E5%2525B7%2525A5%2525E4%2525BD%25259C%2525EF%2525BC%25258C%2525E6%252588%252591%2525E8%2525A6%252581%2525E8%2525B7%25259F%2525E8%252580%252581%2525E6%25259D%2525BF%2525E8%2525B0%252588%2525EF%2525BC%252581%2526linkType%253D&g=%2Fwww.zhipin.com%2F%3Fsid%3Dsem_pz_bdpc_dasou_title&s=3&friend_source=0&s=3&friend_source=0; __c=1606211324; __a=69705857.1606211324..1606211324.22.1.22.22; Hm_lpvt_194df3105ad7148dcf2b98a91b5e727a=1606228354; __zp_stoken__=8d36bZ39zFkJSUkY%2FaykGFHYUeiVKXmxDfg59awlMFGdpb2xxHy8dQ3BaTTlYbmdWPEx%2BCnRQcRxVV19fFkV%2BA3EJM2NQfDpARx0NQ1RFOA4cFA5HMGgtICscEmV9R2FjOh10GUZWDGd2dGA8DQ%3D%3D'}

"""
proxies={
    'https':'123.163.121.119:9999'
    
}#添加代理 某些情况下需要代理
"""

comment_set = {'公司名称':[],'薪资':[],'经验要求':[],'学历要求':[]}

#xpath中取文本  /text() 取直系的文本内容   //text()取其下所有文本内容。
#取属性用  /@属性
#索引语句中的下标是从1开始的

re_now = re.compile(r'(\d{1,2})-(\d{1,2})K')
re_now2 = re.compile(r'·(\d{1,2})薪')


for i in range(1,5):
    page_now = str(i)
    params['page'] = page_now  #变化变量1
    params['ka'] = 'page-'+page_now #变化变量2
    response = requests.get(url=url+page_now,params=params,headers=headers)
    response.encoding = 'utf-8'
    xpath_go = etree.HTML(response.text)  #etree.parse()
    data_set = xpath_go.xpath('//div[@class="job-list"]//li') #取对象根

    #print('第'+str(i+1)+'页评论情况： ')输出情况
    for j in data_set:
        money1 = j.xpath('.//span[@class="red"]/text()')[0]  #薪资
        exp_limit = j.xpath('.//div[@class="job-limit clearfix"]/p/text()[1]')[0]
        xueli = j.xpath('.//div[@class="job-limit clearfix"]/p/text()[2]')[0]
        company_n = j.xpath('.//div[@class="company-text"]//a/text()[1]')[0]
        #short_content1 = j.xpath(".//div[@class='short-content']/text()")[0].strip() 获取短评
        comment_set['公司名称'].append(company_n)
        result = re_now.findall(money1)
        result2 = re_now2.findall(money1)
        if(len(result2)>0):
            result2 = result2[0]
        else:
            result2 = ''
        comment_set['薪资'].append(result[0][0]+' '+result[0][1]+' '+result2)
        comment_set['经验要求'].append(exp_limit)
        comment_set['学历要求'].append(xueli)

        #print(comment_set[-1])  输出情况

datafile = pd.DataFrame(comment_set)
timenow = time.strftime('%Y-%m-%d',time.localtime(time.time()))

#路径+文件名+时间的存储方式（后续可自定义）
datafile.to_csv(save_path + save_folder[0] +'/'+ save_name + "_"+timenow+".csv",index=False,encoding='utf-8_sig') 
datafile.to_excel(save_path + save_folder[0] +'/'+ save_name + "_" + timenow + ".xlsx", index=False, encoding='utf-8_sig')

print(end_said)

已经爬取完毕
